### 1)
- image = cv2.imread('your_image_path.jpg')
- (240, 320, 3)
- Early Assumption, the model could have 4 or 6 classes to  
- The shape is the inputinformation for the network

### 2a) 
for the solution to work with 6 subfolders, adjust the assignment of numerical values to y_train with the filenames, blue, blue_fail, red, red_fail, white, white_fail and add to more number(4 and 5)

In [ ]:
# trainingset
# Initialize empty lists for data and labels
x_train = []
y_train = []

# Define the path to your training dataset directory
training_directory = 'Trainingssets/train_four_label'

# Iterate over subdirectories (classes) in the training dataset directory
for class_name in os.listdir(training_directory):
    class_dir = os.path.join(training_directory, class_name)

    # Iterate over image files in the class directory
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)

        # Load image
        image = cv2.imread(image_path)

        # Normalize pixel values
        image = image / 255.0

        # Append image to data list
        x_train.append(image)

        # Assign numerical label based on filename
        if "blue" in image_name:
            y_train.append(0)
        elif "fail" in image_name:
            y_train.append(1)
        elif "red" in image_name:
            y_train.append(2)
        elif "white" in image_name:
            y_train.append(3)
print("Number of training samples:", len(x_train))
print("Number of training labels:", len(y_train))

In [ ]:
#testset
# testset
# Initialize empty lists for data and labels
x_test = []
y_test = []

# Define the path to your training dataset directory
training_directory = 'Testsets/test_four_label'

# Iterate over subdirectories (classes) in the training dataset directory
for class_name in os.listdir(training_directory):
    class_dir = os.path.join(training_directory, class_name)

    # Iterate over image files in the class directory
    for image_name in os.listdir(class_dir):
        image_path = os.path.join(class_dir, image_name)

        # Load image
        image = cv2.imread(image_path)

        # Normalize pixel values
        image = image / 255.0

        # Append image to data list
        x_test.append(image)

        # Assign numerical label based on filename
        if "blue" in image_name:
            y_test.append(0)
        elif "fail" in image_name:
            y_test.append(1)
        elif "red" in image_name:
            y_test.append(2)
        elif "white" in image_name:
            y_test.append(3)

# Print the loaded data and labels
print("Number of test samples:", len(x_test))
print("Number of test labels:", len(y_test))

### 2b)
The Imagedatagenerator can be configured to automaticly normalize the data, can label based on the folder structure and can augment the data on the fly. The batch can be configured in the Generator.

In [ ]:
#trainingset
# traindirectory
data_dir_train = 'Trainingssets/train_six_label'

# generate Datengeneratoren
datagen = ImageDataGenerator(rescale=1/255.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

# Training Datengenerator
train_generator = datagen.flow_from_directory(
    data_dir_train,
    target_size=(240, 320),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

class_indices = train_generator.class_indices

# Print the class indices
print("Class indices:", class_indices)

In [ ]:
# Testset
# testdirectory
data_dir_eval = 'Testsets/test_six_label'

# configure evaluationgenerator
eval_datagen = ImageDataGenerator(rescale=1/255.)

# create a evaluationset
eval_generator = eval_datagen.flow_from_directory(
    data_dir_eval,
    target_size=(240, 320),  
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)
class_indices = eval_generator.class_indices

# Print the class indices
print("Class indices:", class_indices)

### 3) 
This is just a suggestion, try around to get a good model

In [ ]:
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(240, 320, 3)))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(4, activation='softmax'))

# Modell kompilieren
opt = keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

### 3a)

In [ ]:
#option1
model.fit(x_train, y_train, epochs=10, batch_size=32)

#option2
model.fit(train_generator, epochs=10)

### 3b)
Optimize Options:
- Dataaugmentation -> more Data
- Train longer -> more epochs
- increase the complexity of the network -> more Layers

In [ ]:
#option 1
model.evaluate(x_test, y_test)
#option 2
model.evaluate(eval_generator)

### 4)

In [ ]:
#Dataaugmentation
# ImageDatagenerator
datagen = ImageDataGenerator(rescale=1/255.,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest')

# Keras Layers
model.add(RandomFlip("horizontal"))
model.add(RandomRotation(0.1))
model.add(RandomZoom(0.1))

In [ ]:
#early stopping
early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
model.fit(train_generator, epochs=200, validation_data=eval_generator, callbacks=[early_stopping])
#dropout
model.add(Dropout(0.1))